### Imports

In [1]:
import gurobipy as gp
import pandas as pd

### Functions

In [2]:
def extract(data, row, fact, maximize=True):
    text = str(data.iloc[row][fact]).strip("mg%?")
    if str(text) == "nan" or len(text) == 0:
        return maximize * 1_000_000
    return float(text)

In [3]:
def classify_age(a: int) -> str:
    a = round(a, 0)
    age_strings = ["1-3", "4-8", "9-13", "14-18", "19-30", "31-50", "51-110"]
    age_groups = [tuple([int(age) for age in i.split("-")]) for i in age_strings]
    for i, age in enumerate(age_groups):
        if age[0] <= a <= age[1]:
            return age_strings[i]
    else:
        raise ValueError(f"{a} is not a valid age")

In [4]:
def init_guide_amts(guide):
    print("Converting Percents to Amounts...")
    for col in guide.columns:
        if "% kcal" in col:
            print(col)
            guide[col[:-6] + "AMT"] = pd.Series([0] * len(guide.index))
            for row in range(len(guide.index)):
                cal_levels = [float(i) for i in str(guide.iloc[row]["Calorie Level(s)"]).replace(",", "").split("/")]
                avg_kcals = sum(cal_levels) / len(cal_levels) / 1000
                percents = [float(i) for i in str(guide.iloc[row][col]).strip("< %").replace(",", "").split("-")]
                nut_amt = avg_kcals * sum(percents) / len(percents)
                guide.loc[:, col[:-6] + "AMT"].iloc[row] = nut_amt
    return guide

### Restaurant / Category Selection

In [5]:
def filter_subset(df: pd.DataFrame, restaurant: str, categories: list=[]) -> pd.DataFrame:
    cats = []  # ["Breakfast", "Beverages"]
    subset = df
    subset = subset[subset["Restaurant"] == restaurant]
    if categories != []:
        subset = subset[subset["Common Category"].isin(categories)]
#     if real_meal and not cat_limit:
#         bad_cats = ["Drinks", "Beverages", "Sauces and Condiments", "Coffee", "Salad Dressing"
#                     "Ocean Water", "Iced Teas", "Limeades", "Soft Drinks", ]
#         subset = subset[~subset["Category"].isin(bad_cats)]
    subset.reset_index(inplace=True)
    return subset  # Contains all viable food entries

### File Input

In [6]:

def load_ref_files(nutrition_name="Nutritional Facts", guidelines_name="Dietary Guidelines"):
    df = pd.read_excel(nutrition_name + ".xlsm", index_col=0)
    guide = pd.read_excel(guidelines_name + ".xlsx", header=1, index_col=[1, 2], skiprows=[2]).drop("Unnamed: 0", axis=1)
    guide = init_guide_amts(guide)
    return df, guide

### Requirement Selection

In [7]:
def get_requirements(subset, guide):
    filters = {key: value for key in subset.columns for value in guide.index if "%" not in key and key + ', mg' == value or key + ', g' == value}
    filters.update({"Total Carbohydrates": "Carbohydrate, g", "Sugars": "Added Sugars, AMT"})
    less_thans = ["Sodium", "Sugars"]
    return filters, less_thans

In [8]:
def guide_lookup(gender: str, age: int, guide: pd.DataFrame):
    genders = {"m": "Male", "f": "Female"}
    gender = genders[gender[0].lower()]
#     print(f"Nutritional Requirements for a {age} year old {gender}:")
    return guide.loc[gender, classify_age(age)]

### Model Building: Variables, Constraints, and Objective

In [9]:
def create_model(subset, filters, less_thans, guide, min_cal_cutoff=1, real_meal=True, cat_limit=True, meals=2, verbose=True, filter_relaxations=[]):
    if verbose:
        vprint = print
    else:
        vprint = lambda *x, **y: None
    m = gp.Model()
    xis = [m.addVar(vtype=gp.GRB.BINARY) for _ in subset.index]  # GRB.BINARY / GRB.INTEGER
    f_rel = {fact: m.addVar(name=f"{fact}_rel") if fact in filter_relaxations else 0 for fact in filters}
    for fact, req in filters.items():  # Nutrition Requirements
        if fact in less_thans:
            vprint(f"Constraining {fact}".ljust(35), f"<= {round(guide[filters[fact]], 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) - f_rel[fact] <= float(guide[filters[fact]]) / meals)
        else:
            vprint(f"Constraining {fact}".ljust(35), f">= {round(guide[filters[fact]], 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) >= float(guide[filters[fact]]) / meals)

    if real_meal:
        for i, food in enumerate(subset.iloc):  # Excludes zero calorie (from fat) entries
            m.addConstr(xis[i] <= extract(subset, i, "Calories From Fat", maximize=True) * min_cal_cutoff)

    if cat_limit:
        vprint()
        for cat in pd.unique(subset["Common Category"]):
            vprint(f"Constraining only 1 or fewer {cat.strip('s')} items.")
            m.addConstr(sum([x for i, x in enumerate(xis) if subset["Common Category"][i] == cat]) <= cat_limit)  # TODO: Update to support arbitrary limits
    
    m.setParam("OutputFlag", bool(verbose))
    m.ModelSense = gp.GRB.MINIMIZE
    costs = {"Sugars": 10, "Sodium": 1}
    overage_cost = [f_rel[fact] * costs[fact] if fact in costs else f_rel[fact] for fact in filters]
    m.setObjective(sum((x * extract(subset, r, "Calories From Fat", maximize=True) for r, x in enumerate(xis))) + sum(overage_cost) * 2)
    return m

### Meal Output

In [10]:
def display_details(m, model_args, subset):
    best_choices = [i for i, x in enumerate(m.getVars()) if x.x > 0 and "rel" not in x.VarName]
#     print(sum([m.x[i] for i in best_choices]), "calories from fat")  # TODO: Appears to produce incorrect output...
    
#     print(subset.iloc[best_choices]["Food"], ":")
    selection = subset.iloc[best_choices]
    print(f"The following foods will satisfy your requirements across {model_args['meals']} meals, subject to these overages:", overages)
    display(selection)

### Open Datasources

In [11]:
df, guide = load_ref_files()
age, gender = 25, "Male"
guide = guide_lookup(gender, age, guide)

Converting Percents to Amounts...
Protein, % kcal
Carbohydrate, % kcal
Added Sugars, % kcal

C:\Users\zacha\Anaconda3\envs\NutritionResearch\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)



Total Fat, % kcal
Saturated Fat, % kcal


### Solving

In [12]:
for res in pd.unique(df["Restaurant"]):
    print()
    subset = filter_subset(df, res)
    filters, less_thans = get_requirements(subset, guide)
    filter_relaxations = ["Sugars", "Sodium"]
    model_args = {"subset": subset, "filters": filters, "less_thans": less_thans,
                  "guide": guide, "verbose": False, "filter_relaxations": filter_relaxations,
                  "meals": 2, "cat_limit": 2}
    m = create_model(**model_args)
    m.optimize()
    overages = [x for i, x in enumerate(m.getVars()) if x.x > 0 and "rel" in x.VarName]
    if m.status != 2:
#         raise ValueError("The model is infeasible - please try again")
        print(f"The {res} model is infeasible. Continuing...")
    else:
#         objval = sum([x.x * subset.loc[:, "Calories From Fat"].iloc[i] if "rel" not in x.VarName else 0 for i, x in enumerate(m.getVars())])
        if sum([v.x for v in overages]) == 0:
            print(f"The {res} model is feasible:", "\t" * 5, "<" + "-" * 10)
        else:
            print(f"The {res} model is feasible with the following relaxations:\t\t", overages)
    directions = input(f"'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] ")
    if len(directions) == 0 or directions[0].upper() == "S":
        continue
    if directions[0].upper() == "D":
        display_details(m, model_args, subset)
    if directions[0].upper() == "E":
        break


Using license file C:\Apps\Gurobi\gurobi.lic
Academic license - for non-commercial use only
The Arby's model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 25.0)>, <gurobi.Var Sugars_rel (value 41.86666666666667)>]
'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] D
The following foods will satisfy your requirements across 2 meals, subject to these overages: [<gurobi.Var Sodium_rel (value 25.0)>, <gurobi.Var Sugars_rel (value 41.86666666666667)>]


,index,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,...,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL,Common Category
106,106,Arby's,Breakfast,French Toast Sticks,128g,90,350,10g,1.5g,0.0g,...,15%,8%,0%,10%,19%,20%,14%,0%,https://fastfoodnutrition.org/arbys/french-toa...,Breakfast
125,125,Arby's,Salads,Roast Chicken Salad,1 salad,120,250,14g,7g,0g,...,22%,35%,25%,29%,3%,12%,50%,NaN,https://fastfoodnutrition.org/arbys/roast-chic...,Salads
182,182,Arby's,Sides and Snacks,"Sweet Potato Waffle Fries, kids",68g,120,250,13g,2g,0g,...,20%,10%,0%,4%,10%,16%,2%,NaN,https://fastfoodnutrition.org/arbys/sweet-pota...,Sides
183,183,Arby's,Sides and Snacks,"Sweet Potato Waffle Fries, small",85g,150,310,17g,2.5g,0g,...,26%,13%,0%,5%,13%,20%,4%,NaN,https://fastfoodnutrition.org/arbys/sweet-pota...,Sides



The Baskin-Robbins model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 173.86666666666667)>]
'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] D
The following foods will satisfy your requirements across 2 meals, subject to these overages: [<gurobi.Var Sugars_rel (value 173.86666666666667)>]


,index,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,...,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL,Common Category
7,193,Baskin-Robbins,Ice Cream,"Chocolate Almond Ice Cream, large-scoop",4 oz,160,280,18g,8g,0g,...,28%,40%,15%,4%,8%,12%,14%,8%,https://fastfoodnutrition.org/baskin-robbins/c...,Desserts
42,228,Baskin-Robbins,Ice Cream,"Key Lime Pie Ice Cream, large-scoop",4 oz,100,230,11g,5g,0g,...,17%,25%,7%,6%,12%,12%,8%,8%,https://fastfoodnutrition.org/baskin-robbins/k...,Desserts
155,341,Baskin-Robbins,Beverages,"Tropical Banana Smoothie, medium",24 fl oz,10,540,1g,.5g,0g,...,2%,3%,2%,5%,42%,28%,18%,10%,https://fastfoodnutrition.org/baskin-robbins/t...,Beverages
179,365,Baskin-Robbins,Grab-N-Go,Cake Bites – Chocolate Dipped Strawberry,83 grams,150,290,17g,10.0g,0.0g,...,26%,50%,5%,7%,11%,8%,8%,2%,https://fastfoodnutrition.org/baskin-robbins/c...,To-Go
180,366,Baskin-Robbins,Grab-N-Go,Cake Bites – Love Potion #31®,81 grams,160,300,18g,11.0g,0.0g,...,28%,55%,5%,7%,11%,8%,8%,2%,https://fastfoodnutrition.org/baskin-robbins/c...,To-Go



The Blimpie model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 6.866666666666666)>]
'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] 

The Boston Market model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 645.0)>, <gurobi.Var Sugars_rel (value 11.866666666666667)>]
'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] 

The Buffalo Wild Wings model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 23.866666666666667)>]
'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] 

'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] 
The Burger King model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 715.0)>, <gurobi.Var Sugars_rel (value 26.866666666666667)>]

'S'olve next restaurant; 'E'xit this loop; 'D'etails about current solution; [S/E/D] 